In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import metpy.calc 
from metpy.units import units
import math

C:\Users\josia\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
A105 = pd.read_csv('/Users/josia/Desktop/Python/StickNet/0105A_IOP3_level2.txt')
A105 = A105[106714:111994]
A105

,Time,T,RH,P,WS,WD
106714,2023-03-25 00:30:17,23.9,80.1,998.4,7.6,168.9
106715,2023-03-25 00:30:18,23.9,80.1,998.4,8.7,170.2
106716,2023-03-25 00:30:19,23.9,80.1,998.4,8.4,171.9
106717,2023-03-25 00:30:20,23.9,80.1,998.4,8.6,175.1
106718,2023-03-25 00:30:21,23.9,80.1,998.4,8.5,174.9
...,...,...,...,...,...,...
111989,2023-03-25 01:58:12,23.6,88.3,998.5,12.6,195.0
111990,2023-03-25 01:58:13,23.6,88.3,998.4,13.3,203.1
111991,2023-03-25 01:58:14,23.6,88.3,998.4,12.1,205.1
111992,2023-03-25 01:58:15,23.6,88.2,998.4,12.1,206.5


In [3]:
## Smoothing ##

In [4]:
T = np.array(A105['T'])
RH = np.array(A105['RH'])
P = np.array(A105['P'])
WS = np.array(A105['WS'])
WD = np.array(A105['WD'])

temp = np.zeros(len(T))
rh = np.zeros(len(RH))
p = np.zeros(len(P))
wind_speed = np.zeros(len(WS))
wind_direction = np.zeros(len(WD))

temp[:] = np.nan
rh[:] = np.nan
p[:] = np.nan
wind_speed[:] = np.nan
wind_direction[:] = np.nan

for i in range(2, len(A105) - 2, 1):
    temp[i] = np.nanmean(np.array([T[i-2], T[i-1], T[i], T[i+1], T[i+2]]))
    rh[i] = np.nanmean(np.array([RH[i-2], RH[i-1], RH[i], RH[i+1], RH[i+2]]))
    p[i] = np.nanmean(np.array([P[i-2], P[i-1], P[i], P[i+1], P[i+2]]))
    wind_speed[i] = np.nanmean(np.array([WS[i-2], WS[i-1], WS[i], WS[i+1], WS[i+2]]))
    wind_direction[i] = np.nanmean(np.array([WD[i-2], WD[i-1], WD[i], WD[i+1], WD[i+2]]))

smooth_5s_A105 = pd.DataFrame(data = [np.array(A105['Time']), temp, rh, p, wind_speed, wind_direction], 
                             index = ['Time', 'Temp', 'RH', 'Pressure', 'Wind_Speed', 'Wind_Direction']).transpose()
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN
2,2023-03-25 00:30:19,23.9,80.1,998.4,8.36,172.2
3,2023-03-25 00:30:20,23.9,80.1,998.4,8.48,172.7
4,2023-03-25 00:30:21,23.9,80.1,998.4,8.52,173.04
...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.6,88.3,998.48,12.4,201.48
5276,2023-03-25 01:58:13,23.6,88.28,998.44,12.36,202.64
5277,2023-03-25 01:58:14,23.6,88.26,998.42,12.54,200.68
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN


In [5]:
lon_A105 = -90.5844
lat_A105 = 33.0082
smooth_5s_A105['105A Lon'] = np.full(5280, lon_A105)
smooth_5s_A105['105A Lat'] = np.full(5280, lat_A105)
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082
2,2023-03-25 00:30:19,23.9,80.1,998.4,8.36,172.2,-90.5844,33.0082
3,2023-03-25 00:30:20,23.9,80.1,998.4,8.48,172.7,-90.5844,33.0082
4,2023-03-25 00:30:21,23.9,80.1,998.4,8.52,173.04,-90.5844,33.0082
...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.6,88.3,998.48,12.4,201.48,-90.5844,33.0082
5276,2023-03-25 01:58:13,23.6,88.28,998.44,12.36,202.64,-90.5844,33.0082
5277,2023-03-25 01:58:14,23.6,88.26,998.42,12.54,200.68,-90.5844,33.0082
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082


In [6]:
path = '/Users/josia/Desktop/Python/NSSL Internship/'
meso = pd.read_csv(path + 'Data/mesodata.csv')[0:5280]
smooth_5s_A105['Meso Lon'] = meso['Meso Lon']
smooth_5s_A105['Meso Lat'] = meso['Meso Lat']
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374930,32.679903
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374551,32.679969
2,2023-03-25 00:30:19,23.9,80.1,998.4,8.36,172.2,-90.5844,33.0082,-91.374171,32.680034
3,2023-03-25 00:30:20,23.9,80.1,998.4,8.48,172.7,-90.5844,33.0082,-91.373792,32.680099
4,2023-03-25 00:30:21,23.9,80.1,998.4,8.52,173.04,-90.5844,33.0082,-91.373413,32.680164
...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.6,88.3,998.48,12.4,201.48,-90.5844,33.0082,-90.257489,33.227261
5276,2023-03-25 01:58:13,23.6,88.28,998.44,12.36,202.64,-90.5844,33.0082,-90.257533,33.227391
5277,2023-03-25 01:58:14,23.6,88.26,998.42,12.54,200.68,-90.5844,33.0082,-90.257576,33.227521
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-90.257619,33.227651


In [7]:
dec_time = []
for t in range(0, len(smooth_5s_A105), 1):
    time = smooth_5s_A105['Time'][t]
    hh = int(time[11:13])
    mm = int(time[14:16])
    ss = int(time[17:19])
    dec = hh + ((mm*60) + ss)/3600
    dec_time.append(dec)
smooth_5s_A105['Decimal Time'] = dec_time

In [8]:
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat,Decimal Time
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374930,32.679903,0.504722
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374551,32.679969,0.505000
2,2023-03-25 00:30:19,23.9,80.1,998.4,8.36,172.2,-90.5844,33.0082,-91.374171,32.680034,0.505278
3,2023-03-25 00:30:20,23.9,80.1,998.4,8.48,172.7,-90.5844,33.0082,-91.373792,32.680099,0.505556
4,2023-03-25 00:30:21,23.9,80.1,998.4,8.52,173.04,-90.5844,33.0082,-91.373413,32.680164,0.505833
...,...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.6,88.3,998.48,12.4,201.48,-90.5844,33.0082,-90.257489,33.227261,1.970000
5276,2023-03-25 01:58:13,23.6,88.28,998.44,12.36,202.64,-90.5844,33.0082,-90.257533,33.227391,1.970278
5277,2023-03-25 01:58:14,23.6,88.26,998.42,12.54,200.68,-90.5844,33.0082,-90.257576,33.227521,1.970556
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-90.257619,33.227651,1.970833


In [9]:
## Haversine ##

In [10]:
smooth_5s_A105['105A Lon Rad'] = np.radians(smooth_5s_A105['105A Lon'])
smooth_5s_A105['105A Lat Rad'] = np.radians(np.array(smooth_5s_A105['105A Lat']))
smooth_5s_A105['Meso Lon Rad'] = np.radians(smooth_5s_A105['Meso Lon'])
smooth_5s_A105['Meso Lat Rad'] = np.radians(smooth_5s_A105['Meso Lat'])

smooth_5s_A105['a'] = ((np.sin((smooth_5s_A105['105A Lat Rad'] - smooth_5s_A105['Meso Lat Rad']) / 2))**2 
             + np.cos(smooth_5s_A105['Meso Lat Rad'])*np.cos(smooth_5s_A105['105A Lat Rad'])*(np.sin((smooth_5s_A105['105A Lon Rad'] - smooth_5s_A105['Meso Lon Rad'])/2))**2)

smooth_5s_A105['c'] = 2*np.arctan2(np.sqrt(smooth_5s_A105['a']), np.sqrt((1 - smooth_5s_A105['a'])))

smooth_5s_A105['Distance'] = smooth_5s_A105['c']*6371 # unit for Radius of Earth is km

y = np.sin(smooth_5s_A105['105A Lon Rad'] - (smooth_5s_A105['Meso Lon Rad']))*np.cos(smooth_5s_A105['105A Lat Rad'])
x = ((np.cos(smooth_5s_A105['Meso Lat Rad'])*np.sin(smooth_5s_A105['105A Lat Rad'])) 
     - (np.sin(smooth_5s_A105['Meso Lat Rad'])*np.cos(smooth_5s_A105['105A Lat Rad'])*np.cos(smooth_5s_A105['105A Lon Rad'] - smooth_5s_A105['Meso Lon Rad'])))
smooth_5s_A105['Bearing'] = np.arctan2(y, x)

smooth_5s_A105['Bearing Degrees'] = (np.rad2deg(smooth_5s_A105['Bearing']))

smooth_5s_A105['x'] = smooth_5s_A105['Distance']*np.sin(smooth_5s_A105['Bearing'])
smooth_5s_A105['y'] = smooth_5s_A105['Distance']*np.cos(smooth_5s_A105['Bearing'])

smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat,...,105A Lat Rad,Meso Lon Rad,Meso Lat Rad,a,c,Distance,Bearing,Bearing Degrees,x,y
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374930,32.679903,...,0.576102,-1.594793,0.570372,0.000042,0.012931,82.380893,1.107984,63.482780,73.714441,36.780331
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-91.374551,32.679969,...,0.576102,-1.594787,0.570373,0.000042,0.012925,82.345895,1.107874,63.476479,73.679081,36.772808
2,2023-03-25 00:30:19,23.9,80.1,998.4,8.36,172.2,-90.5844,33.0082,-91.374171,32.680034,...,0.576102,-1.594780,0.570374,0.000042,0.012920,82.310899,1.107763,63.470173,73.643722,36.765286
3,2023-03-25 00:30:20,23.9,80.1,998.4,8.48,172.7,-90.5844,33.0082,-91.373792,32.680099,...,0.576102,-1.594774,0.570375,0.000042,0.012914,82.275904,1.107653,63.463862,73.608363,36.757763
4,2023-03-25 00:30:21,23.9,80.1,998.4,8.52,173.04,-90.5844,33.0082,-91.373413,32.680164,...,0.576102,-1.594767,0.570376,0.000042,0.012909,82.240909,1.107543,63.457544,73.573004,36.750240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.6,88.3,998.48,12.4,201.48,-90.5844,33.0082,-90.257489,33.227261,...,0.576102,-1.575290,0.579925,0.000009,0.006120,38.990599,-2.244017,-128.572678,-30.483548,-24.310906
5276,2023-03-25 01:58:13,23.6,88.28,998.44,12.36,202.64,-90.5844,33.0082,-90.257533,33.227391,...,0.576102,-1.575291,0.579927,0.000009,0.006121,38.996464,-2.244371,-128.593001,-30.479506,-24.325375
5277,2023-03-25 01:58:14,23.6,88.26,998.42,12.54,200.68,-90.5844,33.0082,-90.257576,33.227521,...,0.576102,-1.575292,0.579930,0.000009,0.006122,39.002333,-2.244726,-128.613317,-30.475465,-24.339844
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.5844,33.0082,-90.257619,33.227651,...,0.576102,-1.575293,0.579932,0.000009,0.006123,39.008207,-2.245080,-128.633627,-30.471424,-24.354313


In [11]:
## Variable Calcs ##

In [12]:
theta = []
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    theta_calc = metpy.calc.potential_temperature(p, t)
    theta.append(theta_calc)

In [13]:
td = []
for i in range(0, len(smooth_5s_A105), 1):
    t = smooth_5s_A105['Temp'][i]*units.degC
    rh = (smooth_5s_A105['RH'][i])*units.percent
    td_calc = metpy.calc.dewpoint_from_relative_humidity(t, rh)
    td.append(td_calc)

In [14]:
smooth_5s_A105['Dew Point'] = td

In [15]:
theta_e = []
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    td = smooth_5s_A105['Dew Point'][i]
    theta_e_calc = metpy.calc.equivalent_potential_temperature(p, t, td)
    theta_e.append(theta_e_calc)

In [16]:
#e = []
#for i in range(0, len(data), 1):
#    td = data['Dew Point'][i]
#    e_calc = 611.2*math.exp((17.67*td)/(td + 243.5))
#    e_calc = (e_calc/100)*units.hPa
#    e.append(e_calc)

In [17]:
#mixing_ratio = [] # in kg/kg, unitless
#molecular_weight_ratio = 0.6219569100577033
#for i in range(0, len(data), 1):
#    partial_pressure = e[i]
#    p = data['Pressure'][i]*units.hPa
#    mix_rat_calc = metpy.calc.mixing_ratio(partial_pressure, p, molecular_weight_ratio)
#    mixing_ratio.append(mix_rat_calc)

In [18]:
mixing_ratio = []
for i in range(0, len(smooth_5s_A105), 1):
    t = smooth_5s_A105['Temp'][i]*units.degC
    rh = (smooth_5s_A105['RH'][i])*units.percent
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    mix_rat_calc = metpy.calc.mixing_ratio_from_relative_humidity(p, t, rh)
    mixing_ratio.append(mix_rat_calc)

In [19]:
theta_v = []
molecular_weight_ratio = 0.6219569100577033
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    mix = mixing_ratio[i]
    theta_v_calc = metpy.calc.virtual_potential_temperature(p, t, mix, molecular_weight_ratio)
    theta_v.append(theta_v_calc)

In [20]:
wind_comp = []
for i in range (0, len(smooth_5s_A105), 1):
    spd = smooth_5s_A105['Wind_Speed'][i]*units('m/s')
    dire = smooth_5s_A105['Wind_Direction'][i]*units.deg
    comp = metpy.calc.wind_components(spd, dire)
    wind_comp.append(comp)

In [21]:
u = []
v = []
for i in range(0, len(wind_comp), 1):
    ui, vi = wind_comp[i]
    u.append(ui)
    v.append(vi)

In [22]:
storm_speed = np.full(5280, 23)
storm_motion = np.full(5280, 241)

In [23]:
wind_comp_storm = []
for i in range (0, len(smooth_5s_A105), 1):
    spd = storm_speed[i]*units('m/s')
    dire = storm_motion[i]*units.deg
    comp = metpy.calc.wind_components(spd, dire)
    wind_comp_storm.append(comp)

In [24]:
u_s = []
v_s = []
for i in range(0, len(wind_comp_storm), 1):
    uj, vj = wind_comp_storm[i]
    u_s.append(uj)
    v_s.append(vj)

In [25]:
altimeter = [] # https://www.weather.gov/media/epz/wxcalc/altimeterSetting.pdf
a = 0.190284
b = 0.0065*1013.25**a
for i in range(0, len(smooth_5s_A105), 1):
    alt_set = ((smooth_5s_A105['Pressure'][i] - 0.3)
               *(1 + ((b/288)*((31)/((smooth_5s_A105['Pressure'][i] - 0.3)**a))))**(1/a))
    altimeter.append(alt_set)

In [26]:
mslp = []
for i in range(0, len(smooth_5s_A105)):
    mslp_calc = metpy.calc.altimeter_to_sea_level_pressure(altimeter[i]*units.hPa, 
                                                           31*units.m, 
                                                           smooth_5s_A105['Temp'][i]*units.degC)
    mslp.append(mslp_calc)

In [27]:
delta_d = [np.nan*units.km]
x_bar = [np.nan]
y_bar = [np.nan]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    x0 = smooth_5s_A105['x'][i - 1]
    y0 = smooth_5s_A105['y'][i - 1]
    x2 = smooth_5s_A105['x'][i + 1]
    y2 = smooth_5s_A105['y'][i + 1]
    dist = (((x2 - x0)**2 + (y2 - y0)**2)**(1/2))*units.km
    x = (x2 + x0)/2
    y = (y2 + y0)/2
    delta_d.append(dist)
    x_bar.append(x)
    y_bar.append(y)
delta_d.append(np.nan*units.km)
x_bar.append(np.nan)
y_bar.append(np.nan)

In [28]:
smooth_5s_A105['Theta'] = theta
smooth_5s_A105['Theta_e'] = theta_e
smooth_5s_A105['Theta_v'] = theta_v
#smooth_5s_A105['Vapor Pressure'] = e
smooth_5s_A105['Mixing Ratio'] = mixing_ratio
smooth_5s_A105['u_wind'] = u
smooth_5s_A105['v_wind'] = v
smooth_5s_A105['Storm Speed'] = storm_speed
smooth_5s_A105['Storm Motion'] = storm_motion
smooth_5s_A105['storm_u'] = u_s
smooth_5s_A105['storm_v'] = v_s
smooth_5s_A105['storm_rel_u'] = smooth_5s_A105['u_wind'] - smooth_5s_A105['storm_u']
smooth_5s_A105['storm_rel_v'] = smooth_5s_A105['v_wind'] - smooth_5s_A105['storm_v']
smooth_5s_A105['Altimeter'] = altimeter
smooth_5s_A105['MSLP'] = mslp
smooth_5s_A105['Delta Probe Distance'] = delta_d
smooth_5s_A105['Gradient Mean x'] = x_bar
smooth_5s_A105['Gradient Mean y'] = y_bar

In [29]:
density = []
mol_weight = 0.62195691
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    r = smooth_5s_A105['Mixing Ratio'][i]
    den = metpy.calc.density(p, t, r, mol_weight)
    density.append(den)

In [30]:
## Gradient Calculations ##

In [31]:
grad_theta = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta2 = smooth_5s_A105['Theta'][i + 1]
    theta1 = smooth_5s_A105['Theta'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta = (theta2 - theta1) / d
    grad_theta.append(dtheta)
grad_theta.append(np.nan*units('K/km'))

In [32]:
grad_theta_e = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta_e2 = smooth_5s_A105['Theta_e'][i + 1]
    theta_e1 = smooth_5s_A105['Theta_e'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta_e = (theta_e2 - theta_e1) / d
    grad_theta_e.append(dtheta_e)
grad_theta_e.append(np.nan*units('K/km'))

In [33]:
grad_theta_v = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta_v2 = smooth_5s_A105['Theta_v'][i + 1]
    theta_v1 = smooth_5s_A105['Theta_v'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta_v = (theta_v2 - theta_v1) / d
    grad_theta_v.append(dtheta_v)
grad_theta_v.append(np.nan*units('K/km'))

In [34]:
grad_temp = [np.nan*units('degC/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    temp2 = smooth_5s_A105['Temp'][i + 1]*units.degC
    temp1 = smooth_5s_A105['Temp'][i - 1]*units.degC
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtemp = ((temp2 - temp1) / d)
    grad_temp.append(dtemp)
grad_temp.append(np.nan*units('degC/km'))

In [35]:
grad_press = [np.nan*units('hPa/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    press2 = smooth_5s_A105['Pressure'][i + 1]*units.hPa
    press1 = smooth_5s_A105['Pressure'][i - 1]*units.hPa
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dpress = ((press2 - press1) / d)
    grad_press.append(dpress)
grad_press.append(np.nan*units('hPa/km'))

In [36]:
baro_vort = [np.nan*units('1 / s')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    gen = (((density[i + 1] - density[i - 1])/delta_d[i].to(units.m))
           *grad_press[i].to(units('Pa/m')))*(1/density[i]**2)
    gen = gen*2*units.s
    baro_vort.append(gen.to(units('1 / s')))
baro_vort.append(np.nan*units('1 / s'))

In [37]:
smooth_5s_A105['Density'] = density
smooth_5s_A105['Delta Theta'] = grad_theta
smooth_5s_A105['Delta Theta_e'] = grad_theta_e
smooth_5s_A105['Delta Theta_v'] = grad_theta_v
smooth_5s_A105['Delta Temp'] = grad_temp
smooth_5s_A105['Delta Pressure'] = grad_press
smooth_5s_A105['Baroclinic Generated Vorticity'] = baro_vort

In [38]:
probe_bearing = []
x_comp = []
y_comp = []

probe_bearing.append(np.nan)


for i in range(1, len(smooth_5s_A105) - 1, 1):
    x0 = smooth_5s_A105['x'][i - 1]
    x1 = smooth_5s_A105['x'][i + 1]
    y0 = smooth_5s_A105['y'][i - 1]
    y1 = smooth_5s_A105['y'][i + 1]
    dx = x1 - x0
    dy = y1 - y0
    if (dx < 0) & (dy < 0):
        angle = np.arctan((dx/dy)) + np.pi
        probe_bearing.append(angle)
    elif (dx < 0) & (dy > 0):
        angle = np.arctan((dy/dx)) + ((3*np.pi)/2)
        probe_bearing.append(angle)
    elif (dx > 0) & (dy < 0):
        angle = np.pi - np.arctan((dx/dy))
        probe_bearing.append(angle)
    elif (dx > 0) & (dy > 0):
        angle = (np.pi/2) - np.arctan((dy/dx))
        probe_bearing.append(angle)
    elif (dx == 0) & (dy > 0):
        angle = 0
        probe_bearing.append(angle)
    elif (dx == 0) & (dy < 0):
        angle = np.pi
        probe_bearing.append(angle)
    elif (dx < 0) & (dy == 0):
        angle = (3*np.pi)/2
        probe_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        probe_bearing.append(angle)
probe_bearing.append(np.nan)

for j in range(0, len(smooth_5s_A105) - 1, 1):
    mag = smooth_5s_A105['Baroclinic Generated Vorticity'][j]
    if (mag > 0):
        mag_bear = probe_bearing[j] - np.pi/2
        u = np.abs(mag)*np.sin(mag_bear)
        v = np.abs(mag)*np.cos(mag_bear)
        x_comp.append(u)
        y_comp.append(v)
    elif (mag < 0):
        mag_bear = probe_bearing[j] + np.pi/2
        u = np.abs(mag)*np.sin(mag_bear)
        v = np.abs(mag)*np.cos(mag_bear)
        x_comp.append(u)
        y_comp.append(v)
    else : # mag == 0
        x_comp.append(np.nan)
        y_comp.append(np.nan)

x_comp.append(np.nan)
y_comp.append(np.nan)



In [39]:
smooth_5s_A105['Probe Bearing'] = probe_bearing
smooth_5s_A105['BGV x_comp'] = x_comp
smooth_5s_A105['BGV y_comp'] = y_comp

In [40]:
wind_bearing = []
bgv_bearing = []

for i in range(0, len(smooth_5s_A105), 1):
    u = smooth_5s_A105['storm_rel_u'][i]
    v = smooth_5s_A105['storm_rel_v'][i]
    if (u < 0) & (v < 0):
        angle = np.arctan((u/v)) + np.pi
        wind_bearing.append(angle)
    elif (u < 0) & (v > 0):
        angle = np.arctan((v/u)) + ((3*np.pi)/2)
        wind_bearing.append(angle)
    elif (u > 0) & (v < 0):
        angle = np.pi - np.arctan((u/v))
        wind_bearing.append(angle)
    elif (u > 0) & (v > 0):
        angle = (np.pi/2) - np.arctan((v/u))
        wind_bearing.append(angle)
    elif (u == 0) & (v > 0):
        angle = 0
        wind_bearing.append(angle)
    elif (u == 0) & (v < 0):
        angle = np.pi
        wind_bearing.append(angle)
    elif (u < 0) & (v == 0):
        angle = (3*np.pi)/2
        wind_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        wind_bearing.append(angle)

for i in range(0, len(smooth_5s_A105), 1):
    u = smooth_5s_A105['BGV x_comp'][i]
    v = smooth_5s_A105['BGV y_comp'][i]
    if (u < 0) & (v < 0):
        angle = np.arctan((u/v)) + np.pi
        bgv_bearing.append(angle)
    elif (u < 0) & (v > 0):
        angle = np.arctan((v/u)) + ((3*np.pi)/2)
        bgv_bearing.append(angle)
    elif (u > 0) & (v < 0):
        angle = np.pi - np.arctan((u/v))
        bgv_bearing.append(angle)
    elif (u > 0) & (v > 0):
        angle = (np.pi/2) - np.arctan((v/u))
        bgv_bearing.append(angle)
    elif (u == 0) & (v > 0):
        angle = 0
        bgv_bearing.append(angle)
    elif (u == 0) & (v < 0):
        angle = np.pi
        bgv_bearing.append(angle)
    elif (u < 0) & (v == 0):
        angle = (3*np.pi)/2
        bgv_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        bgv_bearing.append(angle)

        
streamwise_vort_comp = []
crosswise_vort_comp = []

for i in range(0, len(smooth_5s_A105), 1):
    vort_mag = smooth_5s_A105['Baroclinic Generated Vorticity'][i]
    wind_angle = wind_bearing[i]
    bvg_angle = bgv_bearing[i]
    alpha = wind_angle - bvg_angle
    sw = vort_mag*np.cos(alpha)
    cw = vort_mag*np.sin(alpha)
    streamwise_vort_comp.append(sw)
    crosswise_vort_comp.append(cw)

In [41]:
smooth_5s_A105['Wind Bearing from N'] = wind_bearing
smooth_5s_A105['BGV Bearing from N'] = bgv_bearing
smooth_5s_A105['Streamwise Vort Mag'] = streamwise_vort_comp
smooth_5s_A105['Crosswise Vort Mag'] = crosswise_vort_comp

In [42]:
smooth_5s_A105.to_csv('/Users/josia/Desktop/Python/StickNet/105A_Data_Final.csv', index = False)